# Chunk Initial Conditions
Tim Tyree<br>
8.13.2020

In [2]:
from lib.my_initialization import *
from lib.routines.chunk_ic import *
# from lib.controller.controller_LR import get_one_step_explicit_synchronous_splitting
# from lib.model.LR_model import *
# from lib.utils.utils_traj import *
# from lib.routines.bdrates import *
# from lib.measure.utils_measure_tips_cpu import *
# from lib.utils.stack_txt_LR import *
from lib.viewer import *
import trackpy

%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- DONE: use dask to do a run a trivial function with 8 workers an an array of string input that returns True
- DONE: get a list of files to operate on
- DONE: clean up messy stuff left in my folder structure 
- DONE: for the first of ^that list, functionally chunk it into ninths using chunk_600x600_into_ninths(txt,file_name, save_folder)
- TODO: run dask on to create my ic's

In [2]:
# #pylab
# %matplotlib inline
# import numpy as np, pandas as pd, matplotlib.pyplot as plt
# # from pylab import imshow, show
# import time
# # import skimage as sk
# # from skimage import measure, filters

# # import matplotlib.pyplot as plt, numpy as np, os
# # from numba import njit, jit, vectorize
# # from PIL import Image
# # import imageio

# #automate the boring stuff
# from IPython import utils
# import time, os, sys, re
# beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
# if not 'nb_dir' in globals():
#     nb_dir = os.getcwd()

# import dask.bag as db
# from lib.chunk_ic import chunk_600x600_from_fn

# #load the libraries
# from lib import *

# %autocall 1
# %load_ext autoreload
# %autoreload 2

# Goal: use dask to do a run a trivial function with 8 workers an an array of string input that returns True

## dev initial dask bag use

In [27]:
from lib.chunk_ic import chunk_600x600_into_ninths, load_fortran, parse_input_fn

In [18]:
import dask
# !pip install "dask[bag]" --upgrade
import dask.bag as db

In [21]:
#find file interactively
print("please select a file from within the desired folder.")
file = search_for_file()

#get a list of all files
#change to the directory containing file
folder_name = os.path.dirname(file)
os.chdir(folder_name)

#find folder interactively. Crashes jupyter's root kernel :(
# from tkinter import Tk
# from tkinter.filedialog import askdirectory
# path = askdirectory(title='Select Folder') # shows dialog box and return the path
# print(path) 

please select a file from within the desired folder.
File: /Users/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-1/ic-wj/ic_600x600.122


In [22]:
# get all .csv files in the current working directory
retval = !ls
file_name_list = list(retval)
# check each file if it ends in .csv before merging it
def is_csv(file_name):
    return file_name[-4:]=='.csv'
file_name_list = [f for f in file_name_list if not is_csv(f)]
cwd = os.getcwd()
input_fn_list = [cwd+fn for fn in file_name_list]

In [37]:
# use Dask Bag to load your data. This parallelizes the loading step and reduces inter-worker communication:
# triv = lambda object:object
# load_from_filename = triv
load_from_filename = load_fortran
b = db.from_sequence(input_fn_list).map(load_from_filename)
# b = b.map(triv)
retval = list(b)

In [ ]:
# result = b.compute()
# result = list(b)
# b.to_textfiles('/path/to/data/*.json.gz')

In [ ]:
# use Dask Bag to load your data. This parallelizes the loading step and reduces inter-worker communication:
# triv = lambda object:object
# load_from_filename = triv
load_from_filename = load_fortran
b = db.from_sequence(input_fn_list).map(load_from_filename)
# b = b.map(triv)
retval = list(b)

## for the first of ^that list, functionally chunk it into ninths 
using chunk_600x600_into_ninths(txt,file_name, save_folder)

In [41]:
input_fn = '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-1/ic-wj/ic_600x600.122'

In [48]:
from dask.delayed import delayed


In [55]:
#use the loading protocol on one file
# d = delayed(load_fortran)(input_fn)

In [6]:
def chunk_from_fn(input_fn):
    arr = load_fortran(input_fn)
    file_name, input_folder, output_folder, tmp_folder = parse_input_fn(input_fn)
    chunk_600x600_into_ninths(txt=arr,
                              file_name=file_name.replace('600','200'),
                              save_folder=output_folder
                             )
    print(f'done with {input_fn}!')
    return True

In [60]:
chunk_from_fn(input_fn)

True

# fill ic-in with ic-wj chunked into ninths

In [3]:
# get input_fn_list interactively
#find file interactively
print("please select a file from within the desired folder.")
file = search_for_file()

#get a list of all files
#change to the directory containing file
folder_name = os.path.dirname(file)
os.chdir(folder_name)

please select a file from within the desired folder.
File: /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.004


In [4]:
# get all ic files that are in the current working directory that are not .csv's
retval = !ls
file_name_list = list(retval)
# check each file if it ends in .csv before merging it
def is_csv(file_name):
    return file_name[-4:]=='.csv'
file_name_list = [f for f in file_name_list if not is_csv(f)]
cwd = os.getcwd()
input_fn_list = ['/'.join((cwd,fn)) for fn in file_name_list]

In [4]:
# def chunk_600x600_from_fn(input_fn):
# 	input_fn = os.path.abspath(input_fn)
# 	arr = load_fortran(input_fn)
# 	file_name, input_folder, output_folder, tmp_folder = parse_input_fn(input_fn)
# 	chunk_600x600_into_ninths(txt=arr,
# 							  file_name=file_name.replace('600','200'),
# 							  save_folder=output_folder
# 							 )
# 	print(f'done chunking {input_fn}!')
# 	return True

NameError: name 'chunk_600x600_from_fn' is not defined

In [5]:
start = time.time()
b = db.from_sequence(input_fn_list).map(chunk_600x600_from_fn)
retval = list(b)
print(f"run time for chunking: {time.time()-start:.2f} [seconds?]")

done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.002!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.010!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.007!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.012!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.003!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.006!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.008!
done chunking /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ic-wj/ic_600x600.001!
done chunking /home/timothytyree/Documents/GitHu

# chunk my 1800x1800 initial conditions into ninths

part .12 was gotten from, for example, txt[200:400,0:200]

this indicates the first column and the second row

In [33]:
from lib.viewer.TexturePlot import *
from lib.routines.chunk_ic import *

In [42]:
folder_name=f"{nb_dir}/Data/initial-conditions-suite-3-LR"
os.chdir(folder_name)

In [43]:
#get the list of all input 1800x1800 frames
txt_fn_lst=!ls ic1800x1800_tt
txt_fn_lst

['ic1800x1800_t_1210.npz', 'ic1800x1800_t_2830.npz']

In [47]:
# #DONE: verified that each input texture is not stimulated by a semicircle 
# and verified that the initial translational symmetry from 3x3 tiling has died out
# txt=load_buffer('ic1800x1800_tt/'+txt_fn_lst[0])[0]
# fig=show_buffer_LR(txt.astype('float32'))
# plt.show()

arr[0,1] gives the 0th row and the 1st column

In [48]:
count=0
save_folder='ic-tt'
for txt_fn in txt_fn_lst:
    txt_in=load_buffer('ic1800x1800_tt/'+txt_fn)[0]
    count=chunk_1800x1800_into_ninths_npz(txt_in, save_folder,count,typeout='float64')
    print(f'chunked {txt_fn}')

chunked ic1800x1800_t_1210.npz
chunked ic1800x1800_t_2830.npz


In [49]:
beep(3)
print(count)
!ls ic-tt

20
ic001.npz ic004.npz ic007.npz ic010.npz ic013.npz ic016.npz ic019.npz
ic002.npz ic005.npz ic008.npz ic011.npz ic014.npz ic017.npz
ic003.npz ic006.npz ic009.npz ic012.npz ic015.npz ic018.npz


In [56]:
os.chdir(folder_name)
os.chdir('ic-tt')
ic_fn_lst=get_all_files_matching_pattern(os.path.abspath('ic001.npz'), trgt='.npz')
len(ic_fn_lst)

19

In [60]:
#chunk ic-tt to ic-in
def routine(input_file_name):
    txt_in=load_buffer(input_file_name)
    retval=chunk_600x600_into_ninths_npz(txt_in, input_file_name, save_folder='ic-in',typeout='float32')
    return txt_in.shape

In [61]:
os.chdir(folder_name)
start = time.time()
b = db.from_sequence(ic_fn_lst).map(routine)
retval = list(b)
print(f"the run time for chunking was {time.time()-start:.2f} seconds.")

the run time for chunking was 10.06 seconds.


In [63]:
retval[0]

(600, 600, 18)